# Interpreting Optimization Results: A Managerial Skill

This notebook demonstrates how to **interpret optimization results** correctly. Interpretation is a critical managerial skill that determines whether optimization creates value.

Understanding this helps you:
- Know what optimization results actually mean
- Ask the right questions about results
- Evaluate whether results make sense
- Make informed decisions about implementation


## Key Concepts

**What Optimization Results Mean**:
- This is the best solution for your specific objectives and constraints
- It is optimal for the inputs you provided
- It is not necessarily perfect or the only answer
- The solution is only as good as the model and inputs you provided

**Why Interpretation Matters**:
When you run an optimization model, you get a "solution" - specific numbers telling you what to do. However, these numbers don't automatically mean you should implement them. You need to **interpret** the results to understand:
- Whether the solution makes business sense
- What assumptions and tradeoffs were made
- Whether the solution is practical to implement
- How confident you can be in the results

**The Five Key Interpretation Questions**:
1. **Does this make sense?** - Does the recommendation align with what you know about your business?
2. **What tradeoffs were made?** - What did the model prioritize, and what did it sacrifice?
3. **What factors did the model miss?** - Are there important business considerations the model didn't include?
4. **Is this feasible to implement?** - Can you actually do what the model recommends?
5. **How sensitive is this to inputs?** - What happens if your inputs (costs, demands, etc.) are wrong?

**Critical insight**: Interpretation is where optimization succeeds or fails. Good optimization with poor interpretation leads to poor decisions. A perfectly solved optimization problem is useless if you don't understand what the results mean or whether they're appropriate for your situation.


## Scenario: Production Planning Result

Imagine you're a production manager and you've asked your analytics team to optimize your production plan. They've run an optimization model and given you a result.

**The Situation**:
- You produce two products: Product A and Product B
- You have limited capacity and labor hours
- You want to maximize profit
- The optimization model has calculated the "optimal" production quantities

**Optimization Result**: Produce 1,200 units of Product A and 800 units of Product B

**Your task**: Before implementing this recommendation, you need to interpret this result using the framework of questions. Just because the model says this is "optimal" doesn't mean you should blindly follow it. You need to understand what it means and whether it makes sense for your business.

In this notebook, we'll work through each of the five interpretation questions to help you make an informed decision.


## Step 1: Install Required Packages (Colab)

If you're running this notebook in Google Colab, you need to install the `pulp` package first. This cell can be skipped if running locally and the package is already installed.


In [16]:
# Install pulp package (required for optimization)
# This is needed in Google Colab; can be skipped if already installed locally
%pip install pulp -q



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Libraries

Before we can write code to solve optimization problems, we need to tell Python which "libraries" (pre-written code packages) we want to use. Think of libraries as toolboxes - each one contains specialized tools for specific tasks.

- `pandas` (imported as `pd`): Used for working with data tables (like Excel spreadsheets)
- `numpy` (imported as `np`): Used for mathematical calculations
- `matplotlib.pyplot` (imported as `plt`): Used for creating charts and graphs
- `pulp`: This is the optimization library we'll use to solve our optimization problem

The code below imports these libraries so we can use them later.


In [17]:
# Import the libraries we need
# These are like toolboxes - each provides different tools for our work
import pandas as pd          # For working with data tables
import numpy as np           # For mathematical calculations
import matplotlib.pyplot as plt  # For creating charts (we won't use it here, but it's good to have)
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value, PULP_CBC_CMD  # For optimization


## Step 3: Generate an Optimization Result

Before we can interpret an optimization result, we need to create one. In this step, we'll set up a simple production planning problem and solve it.

**What we're doing**:
1. Define the problem parameters (profits, costs, constraints)
2. Set up the optimization model
3. Solve the model to get a result
4. Display the result

**Understanding the code**:
The code below does several things:
- First, it defines the problem data (how much profit each product makes, what constraints we have)
- Then it creates an optimization model using the PuLP library
- It adds an objective (maximize profit) and constraints (capacity, labor, demand limits)
- Finally, it solves the model and extracts the results

Don't worry if you don't understand every line of code - the important part is understanding what the **result** means and how to interpret it, which we'll do in the following steps.


In [18]:
# ============================================================
# STEP 1: Define the problem data
# ============================================================
# These are the inputs to our optimization problem
# Think of these as the "facts" about our business situation

profit_A = 50        # Profit per unit of Product A (in dollars)
profit_B = 40        # Profit per unit of Product B (in dollars)
capacity = 2000      # Maximum total units we can produce
labor_hours = 1500   # Maximum labor hours available
labor_per_A = 0.5    # Labor hours needed per unit of Product A
labor_per_B = 0.8    # Labor hours needed per unit of Product B
demand_A = 1200      # Maximum demand for Product A (we can't sell more than this)
demand_B = 1000      # Maximum demand for Product B

# ============================================================
# STEP 2: Create the optimization model
# ============================================================
# This creates a new optimization problem called "Production"
# LpMaximize means we want to maximize (not minimize) our objective
model = LpProblem("Production", LpMaximize)

# Create decision variables - these are what we're trying to decide
# "How many units of A should we produce?" and "How many units of B?"
# lowBound=0 means we can't produce negative quantities
# cat='Continuous' means we can produce fractional amounts (like 100.5 units)
produce_A = LpVariable("produce_A", lowBound=0, cat='Continuous')
produce_B = LpVariable("produce_B", lowBound=0, cat='Continuous')

# ============================================================
# STEP 3: Define the objective function
# ============================================================
# This is what we want to maximize: total profit
# Total profit = (profit per A × units of A) + (profit per B × units of B)
model += profit_A * produce_A + profit_B * produce_B, "Total_Profit"

# ============================================================
# STEP 4: Add constraints
# ============================================================
# Constraints are the limits we must respect

# Constraint 1: Total production can't exceed capacity
# Units of A + Units of B ≤ 2000
model += produce_A + produce_B <= capacity, "Capacity_Limit"

# Constraint 2: Labor hours used can't exceed available labor
# (0.5 × Units of A) + (0.8 × Units of B) ≤ 1500
model += labor_per_A * produce_A + labor_per_B * produce_B <= labor_hours, "Labor_Limit"

# Constraint 3: Can't produce more than demand for Product A
model += produce_A <= demand_A, "Demand_A_Limit"

# Constraint 4: Can't produce more than demand for Product B
model += produce_B <= demand_B, "Demand_B_Limit"

# ============================================================
# STEP 5: Solve the optimization problem
# ============================================================
# This runs the solver to find the optimal solution
# PULP_CBC_CMD(msg=0) suppresses the solver's detailed output (we don't need to see all that)
# Without this, you'd see lots of technical solver messages
model.solve(PULP_CBC_CMD(msg=0))

# ============================================================
# STEP 6: Extract and display the results
# ============================================================
# After solving, we get the values of our decision variables
result_A = value(produce_A)        # How many units of A to produce
result_B = value(produce_B)        # How many units of B to produce
result_profit = value(model.objective)  # The total profit we'll make

# Display the results in a readable format
print("OPTIMIZATION RESULT:")
print("=" * 60)
print(f"  Produce {result_A:.0f} units of Product A")
print(f"  Produce {result_B:.0f} units of Product B")
print(f"  Total Profit: ${result_profit:,.2f}")
print(f"\nNow let's interpret this result using the framework of questions.")


OPTIMIZATION RESULT:
  Produce 1200 units of Product A
  Produce 800 units of Product B
  Total Profit: $92,000.00

Now let's interpret this result using the framework of questions.


## Step 4: Question 1: Does This Make Sense?

**The Question**: Does this recommendation make sense given what I know about the business?

**Why This Matters**: 
The first thing you should do when you receive an optimization result is ask yourself: "Does this make sense?" This is a sanity check. Even if the math is correct, the result might not align with your business knowledge.

**What to Look For**:
- Does the solution align with basic business logic? (e.g., if Product A is more profitable, does the model recommend producing more of it?)
- Are the quantities reasonable? (e.g., not producing negative amounts, not exceeding obvious limits)
- Do the numbers match what you'd expect given the constraints?

**Example Red Flags**:
- The model recommends producing more than you can sell
- The model recommends focusing on a less profitable product
- The numbers seem way too high or too low compared to what you know about your business

Let's analyze our result to see if it makes sense.


In [19]:
# ============================================================
# QUESTION 1: Does This Make Sense?
# ============================================================
# We're going to check if the optimization result aligns with
# basic business logic and common sense

print("QUESTION 1: Does This Make Sense?")
print("=" * 60)

# ============================================================
# Check 1: Does the result align with profitability?
# ============================================================
# If Product A is more profitable than Product B, we'd expect
# the model to recommend producing more of A (all else equal)
print(f"\nAnalysis:")
print(f"  - Product A profit: ${profit_A} per unit")
print(f"  - Product B profit: ${profit_B} per unit")
print(f"  - Product A is more profitable")
print(f"  - Result: Produce more of A ({result_A:.0f}) than B ({result_B:.0f})")
print(f"\n✓ This makes sense: Focus on more profitable product")

# ============================================================
# Check 2: Are the quantities within reasonable bounds?
# ============================================================
# We need to verify that the recommended quantities don't violate
# obvious constraints like demand limits or total capacity
print(f"\n  - Result A ({result_A:.0f}) is within demand limit ({demand_A})")
print(f"  - Result B ({result_B:.0f}) is within demand limit ({demand_B})")
print(f"  - Total ({result_A + result_B:.0f}) is within capacity ({capacity})")
print(f"\n✓ Quantities are reasonable and feasible")

# ============================================================
# Summary
# ============================================================
print(f"\nInterpretation: The result aligns with business logic and constraints.")


QUESTION 1: Does This Make Sense?

Analysis:
  - Product A profit: $50 per unit
  - Product B profit: $40 per unit
  - Product A is more profitable
  - Result: Produce more of A (1200) than B (800)

✓ This makes sense: Focus on more profitable product

  - Result A (1200) is within demand limit (1200)
  - Result B (800) is within demand limit (1000)
  - Total (2000) is within capacity (2000)

✓ Quantities are reasonable and feasible

Interpretation: The result aligns with business logic and constraints.


## Step 5: Question 2: What Tradeoffs Were Made?

**The Question**: What tradeoffs did the model make? Are those tradeoffs acceptable?

**Why This Matters**:
Optimization models make tradeoffs - they prioritize some things over others. Understanding these tradeoffs helps you evaluate whether the solution is appropriate for your situation.

**What Are Tradeoffs?**
In optimization, you often can't have everything. For example:
- You might maximize profit but sacrifice product diversity
- You might use all your capacity but leave some labor unused
- You might focus on one product but under-serve another market

**What to Look For**:
- Which constraints are "binding" (fully used up)? This tells you what's limiting your solution
- What did the model prioritize? (e.g., profit over product balance)
- What did the model sacrifice? (e.g., producing less of Product B to maximize profit)

**Understanding Constraint Utilization**:
- If a constraint is at 100% utilization, it's "binding" - it's actively limiting your solution
- If a constraint is at 50% utilization, you have slack - you're not using all of that resource
- Binding constraints tell you what tradeoffs the model made

Let's analyze what tradeoffs our model made.


In [20]:
# ============================================================
# QUESTION 2: What Tradeoffs Were Made?
# ============================================================
# We need to understand which constraints are limiting the solution
# and what the model prioritized

print("QUESTION 2: What Tradeoffs Were Made?")
print("=" * 60)

# ============================================================
# Calculate how much of each resource/constraint we're using
# ============================================================
# This helps us see which constraints are "binding" (fully used)
# and which have "slack" (unused capacity)

# Calculate total capacity used (sum of both products)
capacity_used = result_A + result_B

# Calculate total labor hours used
# Formula: (hours per A × units of A) + (hours per B × units of B)
labor_used = labor_per_A * result_A + labor_per_B * result_B

# Calculate utilization percentages
# Utilization = (Amount Used / Total Available) × 100%
capacity_utilization = (capacity_used / capacity) * 100
labor_utilization = (labor_used / labor_hours) * 100

# ============================================================
# Display constraint utilization
# ============================================================
# This shows us which constraints are binding (at or near 100%)
# and which have slack (below 100%)
print(f"\nConstraint Utilization:")
print(f"  Capacity: {capacity_used:.0f} / {capacity} = {capacity_utilization:.1f}%")
print(f"  Labor: {labor_used:.1f} / {labor_hours} = {labor_utilization:.1f}%")
print(f"  Demand A: {result_A:.0f} / {demand_A} = {(result_A/demand_A)*100:.1f}%")
print(f"  Demand B: {result_B:.0f} / {demand_B} = {(result_B/demand_B)*100:.1f}%")

# ============================================================
# Identify the tradeoffs
# ============================================================
# Based on the utilization, we can identify what tradeoffs were made
print(f"\nTradeoffs Identified:")
if capacity_utilization > 95:
    print(f"  - Used most of capacity (focusing on profit)")
if labor_utilization > 95:
    print(f"  - Used most of labor (maximizing productivity)")
if result_B < demand_B * 0.5:
    print(f"  - Produced less of Product B (prioritized more profitable A)")

# ============================================================
# Summary
# ============================================================
print(f"\nInterpretation: The model prioritized profit by focusing on Product A.")
print(f"This tradeoff (less B production) may be acceptable if profit is the main goal.")


QUESTION 2: What Tradeoffs Were Made?

Constraint Utilization:
  Capacity: 2000 / 2000 = 100.0%
  Labor: 1240.0 / 1500 = 82.7%
  Demand A: 1200 / 1200 = 100.0%
  Demand B: 800 / 1000 = 80.0%

Tradeoffs Identified:
  - Used most of capacity (focusing on profit)

Interpretation: The model prioritized profit by focusing on Product A.
This tradeoff (less B production) may be acceptable if profit is the main goal.


## Step 6: Question 3: What Factors Did the Model Miss?

**The Question**: Are there factors the model did not consider that might affect the solution?

**Why This Matters**:
Optimization models are simplifications of reality. They can only include factors that can be:
- Quantified (measured with numbers)
- Included in the model structure
- Known at the time of modeling

Many important business factors don't fit these criteria, but they still matter for decision-making.

**Common Factors Models Miss**:
1. **Customer relationships**: The model doesn't know which customers are most important
2. **Market positioning**: The model doesn't consider long-term market share
3. **Operational complexity**: The model doesn't account for how hard it is to change production
4. **Risk and uncertainty**: The model assumes inputs are known with certainty
5. **Strategic factors**: The model focuses on short-term profit, not long-term strategy

**What to Do**:
- Identify factors the model didn't consider
- Evaluate whether these factors are important for your decision
- Consider whether the model's recommendation needs to be adjusted based on these factors

Let's think about what our model might have missed.


In [21]:
# ============================================================
# QUESTION 3: What Factors Did the Model Miss?
# ============================================================
# We need to think critically about what the model didn't consider
# This is a qualitative analysis - we're thinking about business factors
# that can't easily be put into a mathematical model

print("QUESTION 3: What Factors Did the Model Miss?")
print("=" * 60)

print(f"\nPotential Factors Not Considered:")
print(f"  1. Customer relationships:")
print(f"     - Model focused on A, but B might have important customers")
print(f"     - Low B production might damage relationships")
print(f"     - Some customers might buy both products - reducing B could lose them")
print(f"\n  2. Market positioning:")
print(f"     - Need to maintain presence in both product lines")
print(f"     - Focusing only on A might reduce market share in B")
print(f"     - Competitors might take advantage if you exit Product B market")
print(f"\n  3. Operational considerations:")
print(f"     - Changing production mix might require retooling")
print(f"     - Staff might need retraining")
print(f"     - Supply chain might need adjustment")
print(f"     - Quality might suffer during transition")
print(f"\n  4. Risk and uncertainty:")
print(f"     - Model assumed fixed demand and costs")
print(f"     - Reality is uncertain - demand might change")
print(f"     - Costs might fluctuate")
print(f"     - What if Product A demand drops suddenly?")
print(f"\n  5. Strategic factors:")
print(f"     - Long-term growth vs short-term profit")
print(f"     - Product portfolio balance")
print(f"     - Brand reputation and customer loyalty")
print(f"     - Future market opportunities")

# ============================================================
# Summary
# ============================================================
print(f"\nInterpretation: The model optimized for profit but may have missed")
print(f"strategic, operational, and relationship factors. These should be considered.")
print(f"\nAction: Evaluate whether any of these factors are important enough")
print(f"to adjust the model's recommendation.")

QUESTION 3: What Factors Did the Model Miss?

Potential Factors Not Considered:
  1. Customer relationships:
     - Model focused on A, but B might have important customers
     - Low B production might damage relationships
     - Some customers might buy both products - reducing B could lose them

  2. Market positioning:
     - Need to maintain presence in both product lines
     - Focusing only on A might reduce market share in B
     - Competitors might take advantage if you exit Product B market

  3. Operational considerations:
     - Changing production mix might require retooling
     - Staff might need retraining
     - Supply chain might need adjustment
     - Quality might suffer during transition

  4. Risk and uncertainty:
     - Model assumed fixed demand and costs
     - Reality is uncertain - demand might change
     - Costs might fluctuate
     - What if Product A demand drops suddenly?

  5. Strategic factors:
     - Long-term growth vs short-term profit
     - Produc

## Step 7: Question 4: Is This Feasible to Implement?

**The Question**: Is this solution feasible to implement? Are there practical barriers?

**Why This Matters**:
Even if a solution is mathematically optimal, it might not be practical to implement. Large changes from current operations can be difficult, expensive, or risky.

**What to Consider**:
- **Magnitude of change**: How different is this from what you're doing now?
- **Operational requirements**: What changes are needed in your operations?
- **Resource requirements**: Do you have the resources to make these changes?
- **Time requirements**: How long will it take to implement?
- **Risk**: What could go wrong during implementation?

**Common Implementation Barriers**:
- Large production changes require retooling or retraining
- Supply chain adjustments take time
- Staff resistance to change
- Capital investment needed
- Quality issues during transition

**What to Look For**:
- Percentage change from current production
- Whether changes are gradual or dramatic
- Whether you have the operational flexibility to make these changes

Let's assess whether our solution is feasible to implement.


In [22]:
# ============================================================
# QUESTION 4: Is This Feasible to Implement?
# ============================================================
# We need to compare the recommended solution to current operations
# to see how much change is required

print("QUESTION 4: Is This Feasible to Implement?")
print("=" * 60)

# ============================================================
# Define current production levels
# ============================================================
# In reality, you'd get these from your current operations data
# For this example, we're assuming current production levels
current_A = 600  # Currently producing 600 units of A
current_B = 800  # Currently producing 800 units of B

# ============================================================
# Calculate the changes required
# ============================================================
# Change = Recommended - Current
change_A = result_A - current_A  # How much more/less A to produce
change_B = result_B - current_B  # How much more/less B to produce

# Calculate percentage changes
# Percentage change = (Change / Current) × 100%
# The "if current_A > 0" prevents division by zero
change_pct_A = (change_A / current_A) * 100 if current_A > 0 else 0
change_pct_B = (change_B / current_B) * 100 if current_B > 0 else 0

# ============================================================
# Display current vs recommended
# ============================================================
print(f"\nCurrent Production:")
print(f"  Product A: {current_A} units")
print(f"  Product B: {current_B} units")
print(f"\nRecommended Production:")
# The :+ in the format means show + sign for positive numbers
print(f"  Product A: {result_A:.0f} units (change: {change_A:+.0f}, {change_pct_A:+.1f}%)")
print(f"  Product B: {result_B:.0f} units (change: {change_B:+.0f}, {change_pct_B:+.1f}%)")

# ============================================================
# Assess implementability
# ============================================================
# Large changes (>50%) are typically harder to implement
# abs() gives the absolute value (ignores positive/negative)
print(f"\nImplementability Assessment:")
if abs(change_pct_A) > 50 or abs(change_pct_B) > 50:
    print(f"  ⚠ Large changes required (may be difficult to implement)")
    print(f"  - May require significant operational changes")
    print(f"  - Staff retraining might be needed")
    print(f"  - Equipment adjustments may be necessary")
    print(f"  - Consider phased implementation")
else:
    print(f"  ✓ Moderate changes (likely feasible)")
    print(f"  - Changes are manageable")
    print(f"  - Implementation should be straightforward")

# ============================================================
# Summary
# ============================================================
print(f"\nInterpretation: Consider whether the required changes are practical.")
print(f"Even if optimal, large changes may need to be phased in gradually.")

QUESTION 4: Is This Feasible to Implement?

Current Production:
  Product A: 600 units
  Product B: 800 units

Recommended Production:
  Product A: 1200 units (change: +600, +100.0%)
  Product B: 800 units (change: +0, +0.0%)

Implementability Assessment:
  ⚠ Large changes required (may be difficult to implement)
  - May require significant operational changes
  - Staff retraining might be needed
  - Equipment adjustments may be necessary
  - Consider phased implementation

Interpretation: Consider whether the required changes are practical.
Even if optimal, large changes may need to be phased in gradually.


## Step 8: Question 5: How Sensitive Is This to Inputs?

**The Question**: How sensitive is this solution to changes in inputs? What if inputs are wrong?

**Why This Matters**:
Optimization models use inputs (like profit margins, demand forecasts, costs) that are often uncertain or estimated. If the solution changes dramatically when inputs change slightly, the solution is "sensitive" and you should be cautious.

**What Is Sensitivity?**
Sensitivity measures how much the solution changes when you change the inputs. 
- **High sensitivity**: Small input changes cause large solution changes → Be cautious!
- **Low sensitivity**: Small input changes cause small solution changes → More reliable

**Why Inputs Might Be Wrong**:
- Demand forecasts are uncertain
- Costs might fluctuate
- Profit margins might change
- Constraints might be estimated

**What to Test**:
- What if profits are 10% higher or lower?
- What if demand is different than forecast?
- What if capacity constraints change?

**Interpreting Sensitivity Results**:
- If the solution stays the same when inputs change → **Stable solution** (good!)
- If the solution changes a lot when inputs change → **Sensitive solution** (be careful!)

Let's test how sensitive our solution is to changes in profit margins.


In [23]:
# ============================================================
# QUESTION 5: How Sensitive Is This to Inputs?
# ============================================================
# We'll test what happens to the solution if our profit estimates are wrong
# This is called "sensitivity analysis"

print("QUESTION 5: How Sensitive Is This to Inputs?")
print("=" * 60)

# ============================================================
# Set up sensitivity test
# ============================================================
# We'll test what happens if profits are 90%, 95%, 100%, 105%, or 110% of our estimates
# This simulates what happens if our profit estimates are slightly wrong
profit_variations = [0.9, 0.95, 1.0, 1.05, 1.1]  # 90% to 110% of current profit
sensitivity_results = []  # This list will store our test results

# ============================================================
# Run optimization for each profit scenario
# ============================================================
# The "for" loop runs the optimization multiple times, each time with different profit values
for mult in profit_variations:
    # Calculate test profits (multiply original profits by the multiplier)
    test_profit_A = profit_A * mult  # e.g., if mult=0.9, profit_A becomes 45 instead of 50
    test_profit_B = profit_B * mult
    
    # Create a new optimization model for this test scenario
    model_test = LpProblem("Sensitivity", LpMaximize)
    produce_A_test = LpVariable("produce_A_test", lowBound=0, cat='Continuous')
    produce_B_test = LpVariable("produce_B_test", lowBound=0, cat='Continuous')
    
    # Set up the same model structure but with different profit values
    model_test += test_profit_A * produce_A_test + test_profit_B * produce_B_test, "Total_Profit"
    model_test += produce_A_test + produce_B_test <= capacity, "Capacity_Limit"
    model_test += labor_per_A * produce_A_test + labor_per_B * produce_B_test <= labor_hours, "Labor_Limit"
    model_test += produce_A_test <= demand_A, "Demand_A_Limit"
    model_test += produce_B_test <= demand_B, "Demand_B_Limit"
    
    # Solve this test scenario
    # PULP_CBC_CMD(msg=0) suppresses solver output so we don't see all the technical messages
    model_test.solve(PULP_CBC_CMD(msg=0))
    
    # Store the results for this scenario
    sensitivity_results.append({
        'Profit Multiplier': f"{mult:.2f}x",  # Label like "0.90x" or "1.10x"
        'A': value(produce_A_test),  # How many units of A in this scenario
        'B': value(produce_B_test),  # How many units of B in this scenario
        'Change A': value(produce_A_test) - result_A,  # Difference from original result
        'Change B': value(produce_B_test) - result_B   # Difference from original result
    })

# ============================================================
# Display sensitivity results
# ============================================================
# Convert results to a table (DataFrame) for easy viewing
sensitivity_df = pd.DataFrame(sensitivity_results)
print("\nSensitivity to Profit Changes:")
print("(This shows how the solution changes when profit estimates are wrong)")
display(sensitivity_df.round(1))

# ============================================================
# Assess sensitivity
# ============================================================
# Find the maximum change in either product
# If changes are small, the solution is stable
# If changes are large, the solution is sensitive
max_change_A = abs(sensitivity_df['Change A']).max()  # Largest change in A
max_change_B = abs(sensitivity_df['Change B']).max()  # Largest change in B

print(f"\nSensitivity Assessment:")
# If changes are more than 20% of the original result, it's sensitive
if max_change_A > result_A * 0.2 or max_change_B > result_B * 0.2:
    print(f"  ⚠ Solution is SENSITIVE to input changes")
    print(f"  - Small errors in inputs could significantly change recommendations")
    print(f"  - Be cautious if inputs are uncertain")
    print(f"  - Consider getting more accurate input estimates")
else:
    print(f"  ✓ Solution is relatively STABLE")
    print(f"  - Small input errors won't drastically change recommendations")
    print(f"  - You can be more confident in the solution")

# ============================================================
# Summary
# ============================================================
print(f"\nInterpretation: Consider input uncertainty when evaluating sensitivity.")
print(f"If inputs are uncertain and solution is sensitive, be cautious about implementation.")

QUESTION 5: How Sensitive Is This to Inputs?

Sensitivity to Profit Changes:
(This shows how the solution changes when profit estimates are wrong)


,Profit Multiplier,A,B,Change A,Change B
0,0.90x,1200.0,800.0,0.0,0.0
1,0.95x,1200.0,800.0,0.0,0.0
2,1.00x,1200.0,800.0,0.0,0.0
3,1.05x,1200.0,800.0,0.0,0.0
4,1.10x,1200.0,800.0,0.0,0.0



Sensitivity Assessment:
  ✓ Solution is relatively STABLE
  - Small input errors won't drastically change recommendations
  - You can be more confident in the solution

Interpretation: Consider input uncertainty when evaluating sensitivity.
If inputs are uncertain and solution is sensitive, be cautious about implementation.


## Step 9: Summary: Complete Interpretation

Now that we've worked through all five interpretation questions, let's create a summary that brings everything together. This summary will help you make a final decision about whether to implement, modify, or reject the optimization result.

**The Decision Framework**:
After answering all five questions, you need to decide:
- **Implement**: If all questions have positive answers
- **Modify**: If some concerns can be addressed (e.g., adjust constraints, phase in changes)
- **Reject**: If interpretation reveals major problems that can't be fixed

Let's create a summary table and decision framework.


In [24]:
# ============================================================
# COMPLETE INTERPRETATION SUMMARY
# ============================================================
# This creates a summary table of all our interpretation findings

print("COMPLETE INTERPRETATION SUMMARY")
print("=" * 60)

# ============================================================
# Create summary data
# ============================================================
# This is a Python dictionary - think of it as a table with column names
# Each list contains the values for one column
interpretation_summary = {
    'Question': [  # Column 1: The five questions we asked
        'Does this make sense?',
        'What tradeoffs were made?',
        'What factors did model miss?',
        'Is this feasible to implement?',
        'How sensitive to inputs?'
    ],
    'Answer': [  # Column 2: Our findings for each question
        'Yes - aligns with business logic',
        'Prioritized profit over product balance',
        'Customer relationships, strategy, risk',
        'Depends on change magnitude',
        'Moderate sensitivity to profit changes'
    ],
    'Implication': [  # Column 3: What this means for our decision
        'Result is reasonable',
        'Tradeoff may be acceptable',
        'Consider additional factors',
        'Assess implementation difficulty',
        'Monitor input accuracy'
    ]
}

# ============================================================
# Convert to DataFrame and display
# ============================================================
# DataFrame is a pandas table structure - makes it easy to display as a nice table
summary_df = pd.DataFrame(interpretation_summary)
display(summary_df)

# ============================================================
# Decision Framework
# ============================================================
# This provides a structured way to make a final decision
print("\nDECISION FRAMEWORK:")
print("  Based on interpretation, decide:")
print("  1. Does the result make sense? → If no, investigate")
print("  2. Are tradeoffs acceptable? → If no, adjust objectives")
print("  3. Are missing factors important? → If yes, consider them")
print("  4. Is implementation feasible? → If no, find alternatives")
print("  5. Is sensitivity acceptable? → If no, be cautious")
print("\nFinal Decision: Implement, Modify, or Reject")
print("  - Implement if interpretation is positive")
print("  - Modify if some concerns can be addressed")
print("  - Reject if interpretation reveals major problems")
print("\nRemember: Interpretation is a critical skill. Always question")
print("optimization results before implementing them.")

COMPLETE INTERPRETATION SUMMARY


,Question,Answer,Implication
0,Does this make sense?,Yes - aligns with business logic,Result is reasonable
1,What tradeoffs were made?,Prioritized profit over product balance,Tradeoff may be acceptable
2,What factors did model miss?,"Customer relationships, strategy, risk",Consider additional factors
3,Is this feasible to implement?,Depends on change magnitude,Assess implementation difficulty
4,How sensitive to inputs?,Moderate sensitivity to profit changes,Monitor input accuracy



DECISION FRAMEWORK:
  Based on interpretation, decide:
  1. Does the result make sense? → If no, investigate
  2. Are tradeoffs acceptable? → If no, adjust objectives
  3. Are missing factors important? → If yes, consider them
  4. Is implementation feasible? → If no, find alternatives
  5. Is sensitivity acceptable? → If no, be cautious

Final Decision: Implement, Modify, or Reject
  - Implement if interpretation is positive
  - Modify if some concerns can be addressed
  - Reject if interpretation reveals major problems

Remember: Interpretation is a critical skill. Always question
optimization results before implementing them.


## Summary: Interpreting Optimization Results

**What Optimization Results Actually Mean**:
- The result is the **best solution** for your specific objectives and constraints
- It is **optimal for the inputs you provided** - but inputs might be wrong or incomplete
- It is **not necessarily perfect** or the only answer
- The solution is only as good as the model and data you used

**Why Interpretation Matters**:
Optimization models are powerful tools, but they're not magic. They can only work with:
- What you tell them (objectives and constraints)
- The data you provide (costs, demands, capacities)
- The factors you include in the model

Real business situations are more complex than any model can capture. That's why interpretation is essential.

**The Five-Question Interpretation Framework**:
1. **Does this make sense?** - Business logic check: Does it align with what you know?
2. **What tradeoffs were made?** - Understand priorities: What did the model sacrifice?
3. **What factors did model miss?** - Consider additional factors: What wasn't included?
4. **Is this feasible to implement?** - Practical assessment: Can you actually do this?
5. **How sensitive to inputs?** - Uncertainty assessment: What if inputs are wrong?

**Critical Insights**: 
- **Interpretation is a critical managerial skill** - it's where optimization succeeds or fails
- **Good optimization with poor interpretation leads to poor decisions** - the math can be perfect, but the decision can still be wrong
- **Always question and evaluate results** - don't blindly trust the model
- **The model is a tool, not a decision-maker** - you make the final decision

**Practical Implications**:
- **Use the framework for every optimization result** - make it a habit
- **Don't take results at face value** - always interpret
- **Apply judgment** - combine model results with your business knowledge
- **Interpretation determines value** - optimization only creates value if you interpret correctly

**Remember**: The goal isn't to get a perfect optimization result. The goal is to make better decisions. Interpretation is what connects optimization results to better decisions.
